In [1]:
!pip install -q -e ../python

In [2]:
import json

from explainprompt.logger import Logger, MemoryLogHandler
from explainprompt.model import (
    Prompt, 
    FewShotExamples, 
    PromptExample, 
    Section, 
    ModelResponse)

from explainprompt.jupyter import ExplainPromptWidget

In [3]:
few_shot_examples = FewShotExamples(examples=[
    PromptExample(label="Example 1", sections=[
        Section(type="question", label="Q", content="If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?"),
        Section(type="answer", label="A", content="There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5."),
    ]),
    PromptExample(label="Example 2", sections=[
        Section(type="question", label="Q", content="There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"),
        Section(type="answer", label="A", content="There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6."),
    ]),
    PromptExample(label="Example 3", sections=[
        Section(type="question", label="Q", content="Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?"),
        Section(type="answer", label="A", content="Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39."),
    ]),
    PromptExample(label="Example 4", sections=[
        Section(type="question", label="Q", content="Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"),
        Section(type="answer", label="A", content="Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8."),
    ]),
    PromptExample(label="Example 5", sections=[
        Section(type="question", label="Q", content="Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?"),
        Section(type="answer", label="A", content="Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9."),
    ]),
    PromptExample(label="Example 6", sections=[
        Section(type="question", label="Q", content="There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?"),
        Section(type="answer", label="A", content="There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29."),
    ]),
    PromptExample(label="Example 7", sections=[
        Section(type="question", label="Q", content="Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?"),
        Section(type="answer", label="A", content="Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33."),
    ]),
    PromptExample(label="Example 8", sections=[
        Section(type="question", label="Q", content="Olivia has $23. She bought five bagels for $3 each. How much money does she have left?"),
        Section(type="answer", label="A", content="Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. The answer is 8."),
    ]),
])

logger = Logger()
log_handler = MemoryLogHandler()
logger.add_handler(log_handler)

user_query = Section(type="question", label="Q", content="Grandma Jones baked 5 apple pies for the fireman’s luncheon. She cut each pie into 8 pieces and set the five pies out on the buffet table for the guests to serve themselves. At the end of the evening, after the guests had taken and eaten their pieces of pie, there were 14 pieces of pie remaining. How many pieces were taken by the guests?")

logger.log_response(ModelResponse(step=2, sections=[user_query]))


few_shot_examples.examples[0].step = 3
user_query.step = 4

prompt = Prompt(
    label="Prompt", 
    sections=[
        few_shot_examples,
        user_query,
        Section(type="text", step=4, label="A", content=""),
    ]
)

logger.log_prompt(prompt)

a = ModelResponse(step=5, sections=[
    Section(type="answer", label="A", content="5 pies were baked and cut into 8 pieces each. The 5 pies were then served to the guests. 8 x 5 = 40 pieces of pie. The guests ate 14 pieces of pie. 40 - 14 = 26 pieces of pie were left. The answer is 26."),
])
logger.log_response(a)

messages = list(map(lambda x: x.model_dump(exclude_none=True), log_handler._messages))

In [4]:
widget = ExplainPromptWidget()
widget.data = json.dumps(messages)
widget.theme = 'dark'
widget

ExplainPromptWidget(data='[{"step": 2, "chain": {"sub_chains": []}, "type": "response", "sections": [{"label":…